# Big Data - Proyecto MLLib
# Experimentación con Modelos de Machine Learning

# Inicializar Spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
import time
import json

spark = SparkSession.builder \
    .appName("Hotel Booking Cancellation - Model Experiments") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

25/06/01 18:16:45 WARN Utils: Your hostname, jgasbul-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/06/01 18:16:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/01 18:16:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Cargar datos preprocesados

In [2]:
print("=== CARGANDO DATOS PREPROCESADOS ===")

train_df = spark.read.parquet("train_processed")
val_df = spark.read.parquet("val_processed")

print(f"Datos de entrenamiento: {train_df.count()} registros")
print(f"Datos de validación: {val_df.count()} registros")

=== CARGANDO DATOS PREPROCESADOS ===


Datos de entrenamiento: 44413 registros
Datos de validación: 11118 registros


# Verificar esquema

In [3]:
print("\nEsquema de los datos:")
train_df.printSchema()


Esquema de los datos:
root
 |-- features: vector (nullable = true)
 |-- is_canceled: integer (nullable = true)



# Configurar evaluadores

In [4]:
binary_evaluator = BinaryClassificationEvaluator(
    rawPredictionCol="rawPrediction",
    labelCol="is_canceled",
    metricName="areaUnderROC"
)

multiclass_evaluator = MulticlassClassificationEvaluator(
    predictionCol="prediction",
    labelCol="is_canceled",
    metricName="f1"
)

# Función para evaluar modelo

In [5]:
def evaluate_model(model, train_data, val_data, model_name):
    """
    Evalúa un modelo y retorna métricas completas
    """
    start_time = time.time()
    
    # Entrenar modelo
    fitted_model = model.fit(train_data)
    
    # Predicciones en validación
    predictions = fitted_model.transform(val_data)
    
    # Calcular métricas
    auc = binary_evaluator.evaluate(predictions)
    f1 = multiclass_evaluator.evaluate(predictions)
    
    # Métricas adicionales
    accuracy_evaluator = MulticlassClassificationEvaluator(
        predictionCol="prediction", labelCol="is_canceled", metricName="accuracy"
    )
    accuracy = accuracy_evaluator.evaluate(predictions)
    
    precision_evaluator = MulticlassClassificationEvaluator(
        predictionCol="prediction", labelCol="is_canceled", metricName="weightedPrecision"
    )
    precision = precision_evaluator.evaluate(predictions)
    
    recall_evaluator = MulticlassClassificationEvaluator(
        predictionCol="prediction", labelCol="is_canceled", metricName="weightedRecall"
    )
    recall = recall_evaluator.evaluate(predictions)
    
    training_time = time.time() - start_time
    
    results = {
        "model": model_name,
        "auc": auc,
        "f1": f1,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "training_time": training_time
    }
    
    return results, fitted_model, predictions

# FASE 1: EVALUACIÓN DE MODELOS BASE

In [6]:
print("\n=== FASE 1: EVALUACIÓN DE MODELOS BASE ===")

# Lista para almacenar todos los resultados
all_results = []
fitted_models = {}


=== FASE 1: EVALUACIÓN DE MODELOS BASE ===


# 1. LOGISTIC REGRESSION

In [7]:
print("\n--- EXPERIMENTO 1: LOGISTIC REGRESSION ---")
lr = LogisticRegression(
    featuresCol="features",
    labelCol="is_canceled",
    maxIter=100
)

lr_results, lr_model, lr_predictions = evaluate_model(lr, train_df, val_df, "Logistic Regression")
all_results.append(lr_results)
fitted_models["Logistic Regression"] = lr_model

print(f"AUC: {lr_results['auc']:.4f}")
print(f"F1-Score: {lr_results['f1']:.4f}")
print(f"Accuracy: {lr_results['accuracy']:.4f}")
print(f"Tiempo: {lr_results['training_time']:.2f}s")


--- EXPERIMENTO 1: LOGISTIC REGRESSION ---


AUC: 0.9014
F1-Score: 0.8192
Accuracy: 0.8217
Tiempo: 10.40s


# 2.RANDOM FOREST

In [8]:
print("\n--- EXPERIMENTO 2: RANDOM FOREST ---")
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="is_canceled",
    numTrees=100,
    seed=42
)

rf_results, rf_model, rf_predictions = evaluate_model(rf, train_df, val_df, "Random Forest")
all_results.append(rf_results)
fitted_models["Random Forest"] = rf_model

print(f"AUC: {rf_results['auc']:.4f}")
print(f"F1-Score: {rf_results['f1']:.4f}")
print(f"Accuracy: {rf_results['accuracy']:.4f}")
print(f"Tiempo: {rf_results['training_time']:.2f}s")


--- EXPERIMENTO 2: RANDOM FOREST ---


AUC: 0.8670
F1-Score: 0.7517
Accuracy: 0.7750
Tiempo: 7.85s


# 3. GRADIENT BOOSTED TREES

In [9]:
print("\n--- EXPERIMENTO 3: GRADIENT BOOSTED TREES ---")
gbt = GBTClassifier(
    featuresCol="features",
    labelCol="is_canceled",
    maxIter=100,
    seed=42
)

gbt_results, gbt_model, gbt_predictions = evaluate_model(gbt, train_df, val_df, "Gradient Boosted Trees")
all_results.append(gbt_results)
fitted_models["Gradient Boosted Trees"] = gbt_model

print(f"AUC: {gbt_results['auc']:.4f}")
print(f"F1-Score: {gbt_results['f1']:.4f}")
print(f"Accuracy: {gbt_results['accuracy']:.4f}")
print(f"Tiempo: {gbt_results['training_time']:.2f}s")


--- EXPERIMENTO 3: GRADIENT BOOSTED TREES ---
AUC: 0.9249
F1-Score: 0.8461
Accuracy: 0.8477
Tiempo: 49.79s


# 4. DECISION TREE

In [10]:
print("\n--- EXPERIMENTO 4: DECISION TREE ---")
dt = DecisionTreeClassifier(
    featuresCol="features",
    labelCol="is_canceled",
    seed=42
)

dt_results, dt_model, dt_predictions = evaluate_model(dt, train_df, val_df, "Decision Tree")
all_results.append(dt_results)
fitted_models["Decision Tree"] = dt_model

print(f"AUC: {dt_results['auc']:.4f}")
print(f"F1-Score: {dt_results['f1']:.4f}")
print(f"Accuracy: {dt_results['accuracy']:.4f}")
print(f"Tiempo: {dt_results['training_time']:.2f}s")


--- EXPERIMENTO 4: DECISION TREE ---
AUC: 0.7781
F1-Score: 0.8055
Accuracy: 0.8090
Tiempo: 1.65s


# 5. NAIVE BAYES

In [11]:
#print("\n--- EXPERIMENTO 5: NAIVE BAYES ---")
#nb = NaiveBayes(
#    featuresCol="features",
#    labelCol="is_canceled"
#)

#nb_results, nb_model, nb_predictions = evaluate_model(nb, train_df, val_df, "Naive Bayes")
#all_results.append(nb_results)
#fitted_models["Naive Bayes"] = nb_model

#print(f"AUC: {nb_results['auc']:.4f}")
#print(f"F1-Score: {nb_results['f1']:.4f}")
#print(f"Accuracy: {nb_results['accuracy']:.4f}")
#print(f"Tiempo: {nb_results['training_time']:.2f}s")
print("\n--- EXPERIMENTO 5: NAIVE BAYES ---")
print("⚠️ NOTA: Naive Bayes requiere características no-negativas.")
print("Nuestro StandardScaler genera valores negativos.")
print("Excluyendo Naive Bayes del análisis por incompatibilidad técnica.")

# Resultado placeholder para mantener consistencia
nb_results = {
    "model": "Naive Bayes",
    "auc": float('nan'),
    "f1": float('nan'), 
    "accuracy": float('nan'),
    "precision": float('nan'),
    "recall": float('nan'),
    "training_time": 0.0,
    "status": "Skipped - Incompatible with StandardScaler"
}

all_results.append(nb_results)
fitted_models["Naive Bayes"] = None
print("✅ Naive Bayes excluido. Continuando con 4 algoritmos.")


--- EXPERIMENTO 5: NAIVE BAYES ---
⚠️ NOTA: Naive Bayes requiere características no-negativas.
Nuestro StandardScaler genera valores negativos.
Excluyendo Naive Bayes del análisis por incompatibilidad técnica.
✅ Naive Bayes excluido. Continuando con 4 algoritmos.


# RANKING DE MODELOS BASE

In [12]:
print("\n=== RANKING DE MODELOS BASE ===")

# Ordenar por F1-Score
sorted_results = sorted(all_results, key=lambda x: x['f1'], reverse=True)

print("\nRanking por F1-Score:")
for i, result in enumerate(sorted_results, 1):
    print(f"{i}. {result['model']}: F1={result['f1']:.4f}, AUC={result['auc']:.4f}")

# Seleccionar automáticamente los top 3
top_3_models = sorted_results[:3]
print(f"\n=== SELECCIONADOS PARA OPTIMIZACIÓN (TOP 3) ===")
for i, result in enumerate(top_3_models, 1):
    print(f"{i}. {result['model']}: F1={result['f1']:.4f}")


=== RANKING DE MODELOS BASE ===

Ranking por F1-Score:
1. Gradient Boosted Trees: F1=0.8461, AUC=0.9249
2. Logistic Regression: F1=0.8192, AUC=0.9014
3. Decision Tree: F1=0.8055, AUC=0.7781
4. Random Forest: F1=0.7517, AUC=0.8670
5. Naive Bayes: F1=nan, AUC=nan

=== SELECCIONADOS PARA OPTIMIZACIÓN (TOP 3) ===
1. Gradient Boosted Trees: F1=0.8461
2. Logistic Regression: F1=0.8192
3. Decision Tree: F1=0.8055


# FASE 2: OPTIMIZACIÓN CON GRID SEARCH

In [13]:
print("\n=== FASE 2: OPTIMIZACIÓN CON GRID SEARCH ===")


=== FASE 2: OPTIMIZACIÓN CON GRID SEARCH ===


# Metodo para Crear un grid de parámetros específico para cada tipo de modelo

In [14]:
def create_param_grid(model_name, model_obj):
    """
    Crea grid de parámetros específico para cada tipo de modelo
    """
    if "Logistic Regression" in model_name:
        return ParamGridBuilder() \
            .addGrid(model_obj.regParam, [0.01, 0.1, 1.0]) \
            .addGrid(model_obj.elasticNetParam, [0.0, 0.5, 1.0]) \
            .addGrid(model_obj.maxIter, [100, 200]) \
            .build()
    
    elif "Random Forest" in model_name:
        return ParamGridBuilder() \
            .addGrid(model_obj.numTrees, [50, 100, 200]) \
            .addGrid(model_obj.maxDepth, [5, 10, 15]) \
            .addGrid(model_obj.minInstancesPerNode, [1, 5, 10]) \
            .build()
    
    elif "Gradient Boosted Trees" in model_name:
        return ParamGridBuilder() \
            .addGrid(model_obj.maxIter, [50, 100, 150]) \
            .addGrid(model_obj.maxDepth, [3, 5, 7]) \
            .addGrid(model_obj.stepSize, [0.01, 0.1, 0.2]) \
            .build()
    
    elif "Decision Tree" in model_name:
        return ParamGridBuilder() \
            .addGrid(model_obj.maxDepth, [5, 10, 15, 20]) \
            .addGrid(model_obj.minInstancesPerNode, [1, 5, 10, 20]) \
            .addGrid(model_obj.impurity, ["gini", "entropy"]) \
            .build()
    
    elif "Naive Bayes" in model_name:
        return ParamGridBuilder() \
            .addGrid(model_obj.smoothing, [0.1, 1.0, 2.0, 5.0]) \
            .build()
    
    else:
        raise ValueError(f"Tipo de modelo no reconocido: {model_name}")

# Metodo para optimizar un modelo específico usando Grid Search

In [15]:
def optimize_model(model_name, original_result):
    """
    Optimiza un modelo específico usando Grid Search
    """
    print(f"\n--- OPTIMIZANDO: {model_name.upper()} ---")
    
    # Crear nuevo modelo del mismo tipo
    if "Logistic Regression" in model_name:
        model_obj = LogisticRegression(featuresCol="features", labelCol="is_canceled", maxIter=100)
    elif "Random Forest" in model_name:
        model_obj = RandomForestClassifier(featuresCol="features", labelCol="is_canceled", numTrees=100, seed=42)
    elif "Gradient Boosted Trees" in model_name:
        model_obj = GBTClassifier(featuresCol="features", labelCol="is_canceled", maxIter=100, seed=42)
    elif "Decision Tree" in model_name:
        model_obj = DecisionTreeClassifier(featuresCol="features", labelCol="is_canceled", seed=42)
    elif "Naive Bayes" in model_name:
        model_obj = NaiveBayes(featuresCol="features", labelCol="is_canceled")
    
    # Crear grid de parámetros
    param_grid = create_param_grid(model_name, model_obj)
    
    # Configurar Cross Validator
    cv = CrossValidator(
        estimator=model_obj,
        estimatorParamMaps=param_grid,
        evaluator=multiclass_evaluator,  # F1-score
        numFolds=3,
        seed=42
    )
    
    print(f"Ejecutando Grid Search con {len(param_grid)} combinaciones...")
    
    # Entrenar
    start_time = time.time()
    cv_model = cv.fit(train_df)
    optimization_time = time.time() - start_time
    
    # Evaluar mejor modelo
    best_predictions = cv_model.transform(val_df)
    best_auc = binary_evaluator.evaluate(best_predictions)
    best_f1 = multiclass_evaluator.evaluate(best_predictions)
    
    # Calcular métricas adicionales
    accuracy_evaluator = MulticlassClassificationEvaluator(
        predictionCol="prediction", labelCol="is_canceled", metricName="accuracy"
    )
    best_accuracy = accuracy_evaluator.evaluate(best_predictions)
    
    precision_evaluator = MulticlassClassificationEvaluator(
        predictionCol="prediction", labelCol="is_canceled", metricName="weightedPrecision"
    )
    best_precision = precision_evaluator.evaluate(best_predictions)
    
    recall_evaluator = MulticlassClassificationEvaluator(
        predictionCol="prediction", labelCol="is_canceled", metricName="weightedRecall"
    )
    best_recall = recall_evaluator.evaluate(best_predictions)
    
    # Calcular mejoras
    original_f1 = original_result['f1']
    f1_improvement = best_f1 - original_f1
    improvement_percent = (f1_improvement / original_f1) * 100
    
    print(f"F1-Score original: {original_f1:.4f}")
    print(f"F1-Score optimizado: {best_f1:.4f}")
    print(f"Mejora: {f1_improvement:+.4f} ({improvement_percent:+.1f}%)")
    print(f"AUC: {best_auc:.4f}")
    print(f"Tiempo optimización: {optimization_time:.1f}s")
    
    # Mostrar mejores parámetros
    best_params = cv_model.bestModel.extractParamMap()
    print("Mejores parámetros:")
    for param, value in best_params.items():
        if hasattr(param, 'name'):
            print(f"  {param.name}: {value}")
    
    # Crear resultado
    optimized_result = {
        "model": f"{model_name} (Grid Search)",
        "auc": best_auc,
        "f1": best_f1,
        "accuracy": best_accuracy,
        "precision": best_precision,
        "recall": best_recall,
        "training_time": optimization_time,
        "f1_improvement": f1_improvement,
        "improvement_percent": improvement_percent,
        "best_params": {param.name: value for param, value in best_params.items() if hasattr(param, 'name')}
    }
    
    return optimized_result, cv_model

# Optimizar los top 3 modelos

In [16]:
optimized_results = []
optimized_models = {}

for result in top_3_models:
    model_name = result['model']
    try:
        opt_result, opt_model = optimize_model(model_name, result)
        optimized_results.append(opt_result)
        optimized_models[model_name] = opt_model
        all_results.append(opt_result)
    except Exception as e:
        print(f"Error optimizando {model_name}: {e}")


--- OPTIMIZANDO: GRADIENT BOOSTED TREES ---
Ejecutando Grid Search con 27 combinaciones...


25/06/01 18:24:27 WARN DAGScheduler: Broadcasting large task binary with size 1002.1 KiB
25/06/01 18:24:27 WARN DAGScheduler: Broadcasting large task binary with size 1002.6 KiB
25/06/01 18:24:27 WARN DAGScheduler: Broadcasting large task binary with size 1003.2 KiB
25/06/01 18:24:27 WARN DAGScheduler: Broadcasting large task binary with size 1004.1 KiB
25/06/01 18:24:27 WARN DAGScheduler: Broadcasting large task binary with size 1005.9 KiB
25/06/01 18:24:27 WARN DAGScheduler: Broadcasting large task binary with size 1008.6 KiB
25/06/01 18:24:28 WARN DAGScheduler: Broadcasting large task binary with size 1013.8 KiB
25/06/01 18:24:28 WARN DAGScheduler: Broadcasting large task binary with size 1016.6 KiB
25/06/01 18:24:28 WARN DAGScheduler: Broadcasting large task binary with size 1017.1 KiB
25/06/01 18:24:28 WARN DAGScheduler: Broadcasting large task binary with size 1017.7 KiB
25/06/01 18:24:28 WARN DAGScheduler: Broadcasting large task binary with size 1018.6 KiB
25/06/01 18:24:28 WAR

F1-Score original: 0.8461
F1-Score optimizado: 0.8581
Mejora: +0.0119 (+1.4%)
AUC: 0.9369
Tiempo optimización: 3426.8s
Mejores parámetros:
  cacheNodeIds: False
  checkpointInterval: 10
  featureSubsetStrategy: all
  featuresCol: features
  impurity: variance
  labelCol: is_canceled
  leafCol: 
  lossType: logistic
  maxBins: 32
  maxDepth: 7
  maxIter: 150
  maxMemoryInMB: 256
  minInfoGain: 0.0
  minInstancesPerNode: 1
  minWeightFractionPerNode: 0.0
  predictionCol: prediction
  probabilityCol: probability
  rawPredictionCol: rawPrediction
  seed: 42
  stepSize: 0.2
  subsamplingRate: 1.0
  validationTol: 0.01

--- OPTIMIZANDO: LOGISTIC REGRESSION ---
Ejecutando Grid Search con 18 combinaciones...
F1-Score original: 0.8192
F1-Score optimizado: 0.8200
Mejora: +0.0007 (+0.1%)
AUC: 0.9000
Tiempo optimización: 99.5s
Mejores parámetros:
  aggregationDepth: 2
  elasticNetParam: 0.0
  family: auto
  featuresCol: features
  fitIntercept: True
  labelCol: is_canceled
  maxBlockSizeInMB: 0.0


# COMPARACIÓN FINAL: TODOS LOS MODELOS

In [17]:
print("\n=== COMPARACIÓN FINAL: TODOS LOS MODELOS ===")

# Tabla comparativa
print("\n{:<35} {:<8} {:<8} {:<8} {:<10} {:<12}".format(
    "Modelo", "F1-Score", "AUC", "Accuracy", "Mejora F1", "Tiempo(s)"))
print("-" * 85)

# Modelos base
for result in all_results:
    if "Grid Search" not in result["model"]:
        print("{:<35} {:<8.4f} {:<8.4f} {:<8.4f} {:<10} {:<12.2f}".format(
            result['model'], result['f1'], result['auc'], 
            result['accuracy'], "-", result['training_time']))

print("-" * 85)

# Modelos optimizados
for result in all_results:
    if "Grid Search" in result["model"]:
        improvement = f"+{result['f1_improvement']:.4f}" if 'f1_improvement' in result else "N/A"
        print("{:<35} {:<8.4f} {:<8.4f} {:<8.4f} {:<10} {:<12.2f}".format(
            result['model'], result['f1'], result['auc'], 
            result['accuracy'], improvement, result['training_time']))

# Encontrar el mejor modelo absoluto
best_model_overall = max(all_results, key=lambda x: x['f1'])

print(f"\n=== MODELO GANADOR ===")
print(f"🏆 {best_model_overall['model']}")
print(f"📊 F1-Score: {best_model_overall['f1']:.4f}")
print(f"📊 AUC: {best_model_overall['auc']:.4f}")
print(f"📊 Accuracy: {best_model_overall['accuracy']:.4f}")

if 'f1_improvement' in best_model_overall:
    print(f"📈 Mejora vs base: +{best_model_overall['f1_improvement']:.4f} ({best_model_overall['improvement_percent']:+.1f}%)")


=== COMPARACIÓN FINAL: TODOS LOS MODELOS ===

Modelo                              F1-Score AUC      Accuracy Mejora F1  Tiempo(s)   
-------------------------------------------------------------------------------------
Logistic Regression                 0.8192   0.9014   0.8217   -          10.40       
Random Forest                       0.7517   0.8670   0.7750   -          7.85        
Gradient Boosted Trees              0.8461   0.9249   0.8477   -          49.79       
Decision Tree                       0.8055   0.7781   0.8090   -          1.65        
Naive Bayes                         nan      nan      nan      -          0.00        
-------------------------------------------------------------------------------------
Gradient Boosted Trees (Grid Search) 0.8581   0.9369   0.8588   +0.0119    3426.85     
Logistic Regression (Grid Search)   0.8200   0.9000   0.8229   +0.0007    99.48       
Decision Tree (Grid Search)         0.8387   0.8166   0.8403   +0.0333    164.84    

# GUARDAR MEJOR MODELO

In [18]:
print("\n=== GUARDAR MEJOR MODELO ===")

# Determinar qué modelo guardar
best_model_name = best_model_overall['model']

if "Grid Search" in best_model_name:
    # Es un modelo optimizado
    base_name = best_model_name.replace(" (Grid Search)", "")
    if base_name in optimized_models:
        best_fitted_model = optimized_models[base_name].bestModel
        print(f"Guardando modelo optimizado: {best_model_name}")
    else:
        print(f"Error: No se encontró el modelo optimizado para {base_name}")
        best_fitted_model = None
else:
    # Es un modelo base
    if best_model_name in fitted_models:
        best_fitted_model = fitted_models[best_model_name]
        print(f"Guardando modelo base: {best_model_name}")
    else:
        print(f"Error: No se encontró el modelo base para {best_model_name}")
        best_fitted_model = None

if best_fitted_model:
    best_fitted_model.write().overwrite().save("best_model")
    print("✅ Mejor modelo guardado en 'best_model/'")
else:
    print("❌ Error: No se pudo guardar el modelo")


=== GUARDAR MEJOR MODELO ===
Guardando modelo optimizado: Gradient Boosted Trees (Grid Search)
✅ Mejor modelo guardado en 'best_model/'


# ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS

In [19]:
print("\n=== ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS ===")

# Analizar importancia si el modelo lo soporta
if hasattr(best_fitted_model, 'featureImportances') and best_fitted_model.featureImportances is not None:
    feature_importances = best_fitted_model.featureImportances.toArray()
    
    print(f"\nTop 10 características más importantes ({best_model_name}):")
    importance_pairs = [(i, importance) for i, importance in enumerate(feature_importances)]
    importance_pairs.sort(key=lambda x: x[1], reverse=True)
    
    for i, (feature_idx, importance) in enumerate(importance_pairs[:10]):
        print(f"{i+1:2d}. Característica {feature_idx:3d}: {importance:.4f}")
else:
    print(f"El modelo {best_model_name} no soporta análisis de importancia de características")


=== ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS ===

Top 10 características más importantes (Gradient Boosted Trees (Grid Search)):
 1. Característica 209: 0.1294
 2. Característica 210: 0.0932
 3. Característica   0: 0.0786
 4. Característica   1: 0.0781
 5. Característica  11: 0.0487
 6. Característica  25: 0.0398
 7. Característica  13: 0.0397
 8. Característica  16: 0.0348
 9. Característica 185: 0.0344
10. Característica  14: 0.0307


# MATRIZ DE CONFUSIÓN DEL MEJOR MODELO

In [20]:
print("\n=== MATRIZ DE CONFUSIÓN DEL MEJOR MODELO ===")

# Obtener predicciones del mejor modelo
if "Grid Search" in best_model_name:
    base_name = best_model_name.replace(" (Grid Search)", "")
    if base_name in optimized_models:
        best_predictions = optimized_models[base_name].transform(val_df)
    else:
        print("Error obteniendo predicciones del modelo optimizado")
        best_predictions = None
else:
    if best_model_name in fitted_models:
        best_predictions = fitted_models[best_model_name].transform(val_df)
    else:
        print("Error obteniendo predicciones del modelo base")
        best_predictions = None

if best_predictions:
    print("\nMatriz de Confusión:")
    confusion_matrix = best_predictions.groupBy("is_canceled", "prediction").count().orderBy("is_canceled", "prediction")
    confusion_matrix.show()


=== MATRIZ DE CONFUSIÓN DEL MEJOR MODELO ===

Matriz de Confusión:


25/06/01 19:19:39 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
[Stage 91893:===========================>                           (1 + 1) / 2]

+-----------+----------+-----+
|is_canceled|prediction|count|
+-----------+----------+-----+
|          0|       0.0| 5850|
|          0|       1.0|  641|
|          1|       0.0|  929|
|          1|       1.0| 3698|
+-----------+----------+-----+



25/06/01 19:19:40 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
                                                                                

# ESTADÍSTICAS FINALES

In [21]:
print("\n=== ESTADÍSTICAS FINALES ===")
print(f"✅ Modelos evaluados inicialmente: 5")
print(f"✅ Modelos optimizados: {len(optimized_results)}")
print(f"✅ Mejor F1-Score final: {best_model_overall['f1']:.4f}")
print(f"✅ Tiempo total experimentación: {sum(r['training_time'] for r in all_results):.1f}s")

if optimized_results:
    avg_improvement = sum(r['f1_improvement'] for r in optimized_results) / len(optimized_results)
    print(f"✅ Mejora promedio por optimización: +{avg_improvement:.4f}")

# Guardar resultados en JSON
final_results = {
    "methodology": {
        "total_models_evaluated": 5,
        "models_optimized": len(optimized_results),
        "selection_criteria": "Top 3 by F1-Score",
        "optimization_method": "Grid Search with 3-fold CV"
    },
    "all_results": all_results,
    "best_model": best_model_overall,
    "experiment_timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
}

with open("experiment_results.json", "w") as f:
    json.dump(final_results, f, indent=2)

print("\n📁 Todos los resultados guardados en 'experiment_results.json'")

print("\n=== RESUMEN EJECUTIVO ===")
print("🎯 PROCESO COMPLETADO:")
print("   1. ✅ Evaluación objetiva de 5 algoritmos")
print("   2. ✅ Selección automática de top 3 por F1-Score")
print("   3. ✅ Optimización con Grid Search + CV")
print("   4. ✅ Selección final del mejor modelo global")
print(f"🏆 RESULTADO: {best_model_overall['model']} - F1: {best_model_overall['f1']:.4f}")

spark.stop()


=== ESTADÍSTICAS FINALES ===
✅ Modelos evaluados inicialmente: 5
✅ Modelos optimizados: 3
✅ Mejor F1-Score final: 0.8581
✅ Tiempo total experimentación: 3760.9s
✅ Mejora promedio por optimización: +0.0153

📁 Todos los resultados guardados en 'experiment_results.json'

=== RESUMEN EJECUTIVO ===
🎯 PROCESO COMPLETADO:
   1. ✅ Evaluación objetiva de 5 algoritmos
   2. ✅ Selección automática de top 3 por F1-Score
   3. ✅ Optimización con Grid Search + CV
   4. ✅ Selección final del mejor modelo global
🏆 RESULTADO: Gradient Boosted Trees (Grid Search) - F1: 0.8581
